In [ ]:
using DataFrames, Plots
# load data
train = readtable("project_train.csv")
test = readtable("project_test.csv")

# least squares regression

In [ ]:
X = convert(Array, train[:month, :day, :hour, :minute, :second, :pos, :neg])
y = convert(Array, train[:y])
# least squares regression
w = X\y
testX = convert(Array, test[:month, :day, :hour, :minute, :second, :pos, :neg])
testy = convert(Array, test[:y])
test_pred = sign.(X*w)
error_rate = sum(abs.(test_pred - sign.(testy)) / 2.0) / size(testX,1)
squared_error = sum(abs2(test_pred - sign.(testy)))

## ridge regression

In [ ]:
function ridge(lambda::Number)
    part1 = transpose(X)*X
    part1 = part1 + (lambda * eye(size(part1,1)))
    w_ridge = inv(part1) * transpose(X) * y
    pred = X*w_ridge
    squared_error = sum(abs2(pred - sign.(y)))
    return squared_error
end

In [3]:
ls = linspace(0,1,20)
losses = [ridge(a) for a in ls]
min_val, min_index = findmin(losses)
final_lambda = ls[min_index]
plot(ls, losses, label="x-lambda; y-quadratic loss")

2×2 Array{Float64,2}:
 0.3  0.0
 0.0  0.3

In [ ]:
part1 = transpose(X)*X
part1 = part1 + (final_lambda * eye(size(part1,1)))
w_ridge = inv(part1) * transpose(X) * y
test_pred = sign.(X*w_ridge)
error_rate = sum(abs.(test_pred - sign.(testy)) / 2.0) / size(testX,1)
squared_error = sum(abs2(test_pred - sign.(testy)))

# proximal gradient method for quadratic loss and l1 regularizer

In [ ]:
function prox_l1(x::Number, alpha=1)
    if x > alpha
        return x-alpha
    elseif x < -alpha
        return x + alpha
    else
        return 0
    end
end
function prox_l1(x::AbstractArray, alpha=1)
    return reshape(Float64[prox_l1(xi, alpha) for xi in x], size(x))
end

In [ ]:
# proximal gradient method for quadratic loss and l1 regularizer
function proxgrad_quad_l1(X, y; maxiters = 10, stepsize = 1, λ = 1, w = zeros(size(X,2)))
    objval = Float64[]
    for i=1:maxiters
        # gradient step
        g = 2X'*(X*w-y) # compute quadratic gradient
        z = w - stepsize*g
        # prox step
        w = prox_l1(z, stepsize*λ)
        # record objective value
        push!(objval, norm(X*w-y)^2 + norm(w,1))
    end
    return w, objval
end

In [ ]:
w_prox, obj = proxgrad_quad_l1(X, y; maxiters = 50, stepsize = .1)
test_pred = sign.(X*w_prox)
error_rate = sum(abs.(test_pred - sign.(testy)) / 2.0) / size(testX,1)
squared_error = sum(abs2(test_pred - sign.(testy)))